# SHSAT Test Results Preliminary EDA / Cleaning Notebook
[Return to project overview](final_project_overview.ipynb)

### Andrew Larimer, Deepak Nagaraj, Daniel Olmstead, Michael Winton (W207-4-Summer 2018 Final Project)

The [NYC SHSAT Test Results 2017](https://www.kaggle.com/willkoehrsen/nyc-shsat-test-results-2017/home) dataset includes...

[Original Kaggle page](https://www.kaggle.com/passnyc/data-science-for-good/home)

### Aim

Improve the chances of students taking the SHSAT and receiving placements in specialized high schools.

### What is PASSNYC?

PASSNYC is a not-for-profit, volunteer organization dedicated to broadening educational opportunities for New York City's talented underserved students.

Promoting Access to Specialized Schools in New York City ("PASSNYC") will focus our efforts in underperforming areas that are historically underrepresented in SHSAT registration. In so doing, PASSNYC will help pave the path to specialized high schools for a more diverse group of students.

PASSNYC aims to identify talented underserved students within New York City’s underperforming school districts in order to increase the diversity of students taking the Specialized High School Admissions Test.

Here's what PASSNYC does:

* Partner with NYC middle schools and communities in underperforming school districts to increase awareness of SHSAT

* Identify high-performing students through a combination of interviews, standardized test performance, grades, and educator testimonials.

* Prepare scholars for SHSAT by connecting with select mentoring and test prep programs.

* 

### What is SHSAT?

The Specialized High Schools Admissions Test (SHSAT) is an examination administered to eighth and ninth grade students residing in New York City and used to determine admission to all but one of the city's nine Specialized High Schools. In 2008, about 29,000 students took the test, and 6,108 students were offered admission to one of the high schools based on the results.[1] On average, 30,000 students take this exam annually.

### What are Specialized High Schools?

The specialized high schools of New York City are nine selective public high schools, established and run by the New York City Department of Education to serve the needs of academically and artistically gifted students. The Specialized High Schools Admissions Test (SHSAT) examination is required for admission to all the schools except LaGuardia, which requires an audition or portfolio for admission.

### Output

We want to put out a "potential for outreach" for each school.

Past proxies include data on:
* English language learners
* Students with disabilities
* Students on free/reduced lunch
* Students with temporary housing

### Impact

The best solutions will enable PASSNYC to identify the schools where minority and underserved students stand to gain the most from services like after-school programs, test preparation, mentoring, or resources for parents.

### Judgement Criteria

* Performance: Clearly articulate why your solution is effective at tackling the problem.
* Influential: Be easy to understand enable PASSNYC to convince stakeholders where services are needed the most.
* Shareable: PASSNYC works with over 60 partner organizations to offer services such as test preparation, tutoring, mentoring, extracurricular programs, educational consultants, community and student groups, trade associations, and more. Winning submissions will be able to provide convincing insights to a wide subset of these organizations. 

## Reading data

First, let us read the dataset and do some initial exploratory analysis.

In [ ]:
import pandas as pd
import numpy as np

school_df = pd.read_csv('2016_school_explorer.csv')
shsat_df = pd.read_csv('d5_shsat_registrations_and_testers.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
school_df.head(5)

Some of the columns are interesting.

* Adjusted grade, "New?", "Other location code" have NaNs.  Need further exploration.
* SED Code is "State Education Dept Code"
* The following are useful for geographical clustering: Location code, district, latitude, longitude, address, city, zip.
* Grades, Grade Low, Grade High gives a sense of the size of school.
* "Community School?" tells whether the school receives extra support: The program pairs selected schools with community based organizations that focus on social and emotional support and development.
* "Economic Need Index" reflects the socioeconomics of the school population.  It is calculated using the following formula:
$$ENI = TemporaryHousing\% + \frac{1}{2} HRAeligible\% + \frac{1}{2} FreeLunchEligible\% $$
* XXX: "School Income Estimate": Don't know
* ELL (English Language Learner) is a proxy for a non-native student, who may benefit: English-language learners, or ELLs, are students who are unable to communicate fluently or learn effectively in English, who often come from non-English-speaking homes and backgrounds, and who typically require specialized or modified instruction in both the English language and in their academic courses.
* Asian / Black / Hispanic / White: to identify underrepresented races.
* "Student attendance rate" and "Percentage of students chronically absent" can be a proxy for students who are distracted from learning.
* XXX: "Rigorous instruction %": Don't know
* XXX: "Collaborative teachers %": Don't know
* XXX: "Supportive environment": Don't know
* XXX: "Effective school leadership": Don't know
* XXX: "Strong family-community ties": Don't know
* XXX: "Trust": Don't know
* XXX: "Student achievement rating": Don't know
* ELA proficiency (English Language Arts), Math proficiency: we can use both to gauge student performance
* "4s" columns: shows highest performing students across grades and races (4s is state standards, 4 is highest performance)


Let us do some cleanup of the data.  [Source](https://www.kaggle.com/randylaosat/simple-exploratory-data-analysis-passnyc)

In [ ]:
import re

# Remove percent and convert to float
percent_columns = [
    'Percent of Students Chronically Absent',
    'Rigorous Instruction %',
    'Collaborative Teachers %',
    'Supportive Environment %',
    'Effective School Leadership %',
    'Strong Family-Community Ties %',
    'Trust %',
    'Student Attendance Rate',
    'Percent ELL',
    'Percent Asian',
    'Percent Black',
    'Percent Hispanic',
    'Percent Black / Hispanic',
    'Percent White',
]
for col in percent_columns:
    school_df[col] = school_df[col].astype(str).apply(lambda s: float(s.strip('%')) / 100)
# Remove $ and , for income column
school_df['School Income Estimate'] = school_df['School Income Estimate'].astype(str).apply(lambda s: float(re.sub('[$,]', '', s)))

# Having spaces etc. can cause annoying problems: replace with underscores
def sanitize_columns(c):
    c = c.lower()
    c = re.sub('[?,()/]', '', c)
    c = re.sub('[ -]', '_', c)
    c = c.replace('%', 'percent')
    return c
school_df.columns = [sanitize_columns(c) for c in school_df.columns]

Now let us have another look at the data.

In [ ]:
display(school_df)

Let us look at some individual columns which seem to have a lot of "NaN"s.

In [ ]:
nulls_df = school_df.isnull()
display(nulls_df.query('adjusted_grade == False'))

In [ ]:
import matplotlib.pyplot as plt
print(school_df['economic_need_index'].describe())
display(school_df['economic_need_index'].hist())

## NYTimes School Data

We also have [NYTimes data](https://www.kaggle.com/willkoehrsen/nyc-shsat-test-results-2017).  Let us read it and have a first look.

In [ ]:
nyt_df = pd.read_csv('nytdf.csv')

In [ ]:
nyt_df.head(5)

Let us do some cleanup.

In [ ]:
# Remove percent and convert to float
percent_columns = [
    'OffersPerStudent',
    'PctBlackOrHispanic',
]
for col in percent_columns:
    nyt_df[col] = nyt_df[col].astype(str).apply(lambda s: int(s.strip('%')))
nyt_df.columns = [sanitize_columns(c) for c in nyt_df.columns]

In [ ]:
nyt_df.head(5)

In [ ]:
# We tried to obtain enrollment information, but this fails when offersperstudent = 0

# nyt_df['numstudents'] = np.where(nyt_df['offersperstudent'] > 0, 
#                                  nyt_df['numspecializedoffers'] * 100.0 / nyt_df['offersperstudent'],
#                                  0).astype('int')
# display(nyt_df)

## DoE Demographics Data

We do not have school enrollment information above.  We obtained it from [NYC OpenData website](https://data.cityofnewyork.us/Education/2013-2018-Demographic-Snapshot-School/s52a-8aq6).

In [ ]:
doe_school_df = pd.read_csv('doe_demographic_snapshot_school.csv')
doe_school_df.head(5)

We will filter it down to the year we are interested in (2017-18) based on the above SHSAT data, and do some cleanup.  We look at 2017-18 because the test is at the very beginning of the year:

> Registration is September 7-October 12, 2017... In 2017, tests were given October 21, 22; October 29, and November 4.

We can also filter information into Grade 8/9 enrollments:

> All students in grades eight and nine who are current New York City residents are eligible. [Source](https://www.schools.nyc.gov/school-life/learning/testing/specialized-high-school-admissions-test)

In [ ]:
doe_school_df.columns = [sanitize_columns(c) for c in doe_school_df.columns]
shsat_eligible_class_size_df = doe_school_df \
    .query("year == '2017-18'") \
    [['dbn', 'grade_8', 'grade_9']]
shsat_eligible_class_size_df.head(5)

We will now combine the enrollment data with the NYTimes data.

In [ ]:
combined_df = nyt_df.merge(shsat_eligible_class_size_df, on='dbn', how='left')
combined_df.head(5)

Let us run some quick sanity checks.

In [ ]:
# using the fact that np.nan != np.nan
display(combined_df.query('grade_8 != grade_8 | grade_9 != grade_9'))
display(combined_df.query('numshsattesttakers > grade_8 + grade_9'))

Looks good.  There are no invalid values or empty values.

Let us add a column for fraction of test takers and look at its distribution.

In [ ]:
combined_df['pct_test_takers'] = (combined_df['numshsattesttakers'] * 100 \
                / (combined_df['grade_8'] + combined_df['grade_9'])).astype('int')
combined_df.head(5)

In [ ]:
display(combined_df.hist('pct_test_takers', bins=20))

In [ ]:
display(combined_df.boxplot('pct_test_takers'))

It looks like we have a sharp drop at around 40%.  We can use that as a cutoff, and label any school that has >40% as "successful".

Let us also have a look at how offers per student is spread out.

In [ ]:
display(combined_df.hist('offersperstudent', bins=20))

In [ ]:
combined_df['success_label'] = (combined_df['pct_test_takers'] > 40).astype('int')

In [ ]:
display(combined_df.head())

We will save this csv file out.

In [ ]:
out_df = combined_df[['dbn', 'numshsattesttakers', 'offersperstudent', 'pct_test_takers', 'success_label']]
out_df.to_csv('shsat_outcome_data.csv', index=False)